# Temp notebook copy to check the new data release

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats

In [2]:
from skyllh.core.config import Config
cfg = Config()

In [3]:
# from skyllh.datasets.i3.PublicData_10y_ps import create_dataset_collection
from skyllh.datasets.i3.PublicData_14y_ps import create_dataset_collection

In [4]:
dsc = create_dataset_collection(
    cfg=cfg, 
    base_path='/data/user/tkontrimas/datarelease_2025/')

In [6]:
datasets = dsc['IC40', 'IC59', 'IC79', 'IC86_I-XI']

The analysis used for the published PRL results is referred in SkyLLH as "*traditional point-source analysis*" and is pre-defined:

In [7]:
from skyllh.analyses.i3.publicdata_ps.time_integrated_ps import create_analysis

In [9]:
from skyllh.core.source_model import PointLikeSource

In [15]:
# source = PointLikeSource(ra=np.deg2rad(77.35), dec=np.deg2rad(5.7)) # TXS

source = PointLikeSource(ra=np.deg2rad(40.67), dec=np.deg2rad(-0.01)) # NGC1068

In [16]:
ana = create_analysis(cfg=cfg, datasets=datasets, source=source)

100%|██████████████████████████████████████████████████████████████████████████████████████| 176/176 [00:00<00:00, 7134.48it/s]


In [17]:
from skyllh.core.random import RandomStateService
rss = RandomStateService(seed=1)

In [18]:
(ts, x, status) = ana.unblind(minimizer_rss=rss)

In [19]:
print(f'TS = {ts:.3f}')
print(f'ns = {x["ns"]:.2f}')
print(f'gamma = {x["gamma"]:.2f}')

TS = 29.922
ns = 79.38
gamma = 3.11


## Calculating the corresponding flux normalization 

By default the analysis is created with a flux normalization of 1 GeV$^{-1}$s$^{-1}$cm$^{-2}$sr$^{-1}$ (see `refplflux_Phi0` argument of the `create_analysis` method). The analysis instance has the method `calculate_fluxmodel_scaling_factor` that calculates the scaling factor the reference flux normalization has to be multiplied with to represent a given analysis result, i.e. $n_{\text{s}}$ and $\gamma$ value. This function takes the detected mean $n_{\text{s}}$ value as first argument and the list of source parameter values as second argument:

In [20]:
scaling_factor = ana.calculate_fluxmodel_scaling_factor(x['ns'], [x['ns'], x['gamma']])
print(f'Flux scaling factor = {scaling_factor:.3e}')

Flux scaling factor = 2.751e-14


Hence, our result corresponds to a power-law flux of:

In [21]:
print(f'{scaling_factor:.3e}'' (E/1000 GeV)^{-'f'{x["gamma"]:.2f}'+'} 1/(GeV s cm^2 sr)')

2.751e-14 (E/1000 GeV)^{-3.11} 1/(GeV s cm^2 sr)


Evaluating the log-likelihood ratio function
---

Sometimes it is useful to be able to evaluate the log-likelihood ratio function, e.g. for creating a likelihood contour plot. Because SkyLLH's structure is based on the mathematical structure of the likelihood function, the `Analysis` instance has the property `llhratio` which is the class instance of the used log-likelihood ratio function. This instance has the method `evaluate`. The method takes an array of the fit parameter values as argument at which the LLH ratio function will be evaluated. It returns the value of the LLH ratio function at the given point and its gradients w.r.t. the fit parameters.

In our case this is the number of signal events, $n_{\mathrm{s}}$ and the spectral index $\gamma$. If we evaluate the LLH ratio function at the maximum, the gradients should be close to zero.

In [ ]:
help(ana.llhratio.evaluate)

In [ ]:
(llhratio_value, (grad_ns, grad_gamma)) = ana.llhratio.evaluate([14.58, 2.17])
print(f'llhratio_value = {llhratio_value:.3f}')
print(f'grad_ns = {grad_ns:.3f}')
print(f'grad_gamma = {grad_gamma:.3f}')

Using the `evaluate` method of the `LLHRatio` class we can scan the log-likelihood ratio space and create a contour plot showing the best fit and the 68%, 90%, and 95% quantile assuming Wilks-theorem.

In [ ]:
(ns_min, ns_max, ns_step) = (0, 80, 0.5)
(gamma_min, gamma_max, gamma_step) = (1.5, 4.0, 0.1)

ns_edges = np.linspace(ns_min, ns_max, int((ns_max-ns_min)/ns_step)+1)
ns_vals = 0.5*(ns_edges[1:] + ns_edges[:-1])

gamma_edges = np.linspace(gamma_min, gamma_max, int((gamma_max-gamma_min)/gamma_step+1))
gamma_vals = 0.5*(gamma_edges[1:] + gamma_edges[:-1])

delta_ts = np.empty((len(ns_vals), len(gamma_vals)), dtype=np.double)
for (ns_i, ns) in enumerate(ns_vals):
    for (gamma_i, gamma) in enumerate(gamma_vals):

        delta_ts[ns_i, gamma_i] = (
            ana.calculate_test_statistic(llhratio_value, [14.58, 2.17]) -
            ana.calculate_test_statistic(ana.llhratio.evaluate([ns, gamma])[0], [ns, gamma])
        )

# Determine the best fit ns and gamma values from the scan.
index_max = np.argmin(delta_ts)
ns_i_max = int(index_max / len(gamma_vals))
gamma_i_max = index_max % len(gamma_vals)
ns_best = ns_vals[ns_i_max]
gamma_best = gamma_vals[gamma_i_max]

In [ ]:
# Determine the delta lambda value for the 95% quantile assuming a chi-sqaure
# distribution with 2 degrees of freedom (i.e. assuming Wilks theorem).
chi2_68_quantile = scipy.stats.chi2.ppf(0.68, df=2)
chi2_90_quantile = scipy.stats.chi2.ppf(0.90, df=2)
chi2_95_quantile = scipy.stats.chi2.ppf(0.95, df=2)

In [ ]:
from matplotlib.colors import LogNorm
plt.figure(figsize=(8,6))
plt.pcolormesh(gamma_edges, ns_edges, delta_ts, cmap='nipy_spectral')
cbar = plt.colorbar()
cbar.set_label(r'$\Delta$TS')
plt.contour(gamma_vals, ns_vals, delta_ts, [chi2_68_quantile], colors='#FFFFFF')
plt.contour(gamma_vals, ns_vals, delta_ts, [chi2_90_quantile], colors='#AAAAAA')
plt.contour(gamma_vals, ns_vals, delta_ts, [chi2_95_quantile], colors='#444444')
plt.plot(gamma_best, ns_best, marker='x', color='white', ms=10)
plt.xlabel(r'$\gamma$')
plt.ylabel(r'$n_{\mathrm{s}}$')
plt.ylim(ns_min, ns_max)
plt.xlim(gamma_min, gamma_max)

Calculating the significance (local p-value)
---

The significance of the source, i.e. the local p-value, can be calculated by generating the test-statistic distribution of background-only data trials, i.e. for zero injected signal events. SkyLLH provides the helper function ``create_trial_data_file`` to do that:

In [ ]:
from skyllh.core.utils.analysis import create_trial_data_file

In [ ]:
help(create_trial_data_file)

At first we will generate 10k trials and look at the test-statistic distribution. We will time the trial generation using the ``TimeLord`` class.

In [ ]:
from skyllh.core.timing import TimeLord
tl = TimeLord()

In [ ]:
rss = RandomStateService(seed=1)
(_, _, _, trials) = create_trial_data_file(
    ana=ana,
    rss=rss,
    n_trials=1e4,
    mean_n_sig=0,
    pathfilename='/home/mwolf/projects/publicdata_ps/txs_bkg_trails.npy',
    ncpu=8,
    tl=tl)
print(tl)

After generating the background trials, we can histogram the test-statistic values and plot the TS distribution.

In [ ]:
(h, be) = np.histogram(trials['ts'], bins=np.arange(0, np.max(trials['ts'])+0.1, 0.1))
plt.plot(0.5*(be[:-1]+be[1:]), h, drawstyle='steps-mid', label='background')
plt.vlines(ts, 1, np.max(h), label=f'TS(TXS 0506+056)={ts:.3f}')
plt.yscale('log')
plt.xlabel('TS')
plt.ylabel('#trials per bin')
plt.legend()
pass

We can see that the TS value of the unblinded data for TXS is rather large and 10k trials are not enough to calculate a reliable estimate for the p-value. Hence, we will generate a few more trials. SkyLLH provides also a helper function to extend the trial data file we just created. It is called ``extend_trial_data_file``: 

In [ ]:
from skyllh.core.utils.analysis import extend_trial_data_file

In [ ]:
help(extend_trial_data_file)

In [ ]:
tl = TimeLord()
rss = RandomStateService(seed=2)
trials = extend_trial_data_file(
    ana=ana,
    rss=rss,
    n_trials=4e4,
    trial_data=trials,
    pathfilename='/home/mwolf/projects/publicdata_ps/txs_bkg_trails.npy',
    ncpu=8,
    tl=tl)

In [ ]:
print(tl)

The local p-value is defined as the fraction of background trials with TS value greater than the unblinded TS value of the source. 

In [ ]:
minus_log10_pval = -np.log10(len(trials[trials['ts'] > ts]) / len(trials))
print(f'-log10(p_local) = {minus_log10_pval:.2f}')

In [ ]:
(h, be) = np.histogram(trials['ts'], bins=np.arange(0, np.max(trials['ts'])+0.1, 0.1))
plt.plot(0.5*(be[:-1]+be[1:]), h, drawstyle='steps-mid', label='background')
plt.vlines(ts, 1, np.max(h), label=f'TS(TXS 0506+056)={ts:.3f}')
plt.yscale('log')
plt.xlabel('TS')
plt.ylabel('#trials per bin')
plt.legend()
pass

In [ ]:
print(ana.data_list[0].exp)

In [ ]:
for i in range(len(ana.data_list)):
    print(ana.data_list[i].exp)

In [ ]:
import numpy as np

In [ ]:
np.unique(np.concatenate([
        np.linspace(-1., -0.75, 10 + 1),
        np.linspace(-0.75, 0., 15 + 1),
        np.linspace(0., 1., 20 + 1),
    ]))

In [ ]:
np.linspace(-1, 1, 50)

In [ ]:
ic79_exp_orig = np.load('/data/ana/analyses/ps_tracks/version-004-p02/IC79_exp.npy')

In [ ]:
ic79_exp_orig

In [ ]:
energy_bins = np.arange(1., 9.5 + 0.01, 0.125)

plt.hist(ic79_exp_orig['logE'], bins=energy_bins)
plt.yscale('log')